In [3]:
!git clone https://github.com/maszhongming/UniEval.git
!pip install -r UniEval/requirements.txt
  
# this import will not work properly in google Colab, to fix this you need to fix the imports in the metric/evaluator.py file.

fatal: destination path 'UniEval' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5

In [6]:
from sklearn import metrics 
from UniEval.utils import convert_to_json
from UniEval.metric.evaluator import get_evaluator
import nltk
import os
import json

nltk.download('punkt')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
def source_sentences_to_list(path):
  out = []
  with open(path) as f:
    for row in f:
      data = json.loads(row)
      out.append(data["premise"])
      out.append(data["hypothesis"])
  return out

In [8]:
def bart_uniEval_scores(data):
  # Initialize evaluator for a specific task
  evaluator = get_evaluator('summarization')
  return evaluator.evaluate(data, dims=['coherence', 'consistency', 'fluency'], 
                                 overall=True, print_result=False)


def write_to_file(results, path):
  with open(path, "w") as f:
    for item in results:
      json.dump(item, f)
      f.write('\n')

In [9]:
def evaluate_data(src_path, inp_path, out_path):
  # original
  src_list = source_sentences_to_list(src_path)
  # a list of model outputs to be evaluataed
  output_list = source_sentences_to_list(inp_path)
  # Prepare data for pre-trained evaluators
  data = convert_to_json(output_list=output_list, src_list=src_list)
  write_to_file(bart_uniEval_scores(data), out_path)

In [10]:
evaluate_data("/content/drive/MyDrive/thesis/multinli_dev_matched.jsonl",
              "/content/drive/MyDrive/thesis/multinli_dev_matched_gpt.jsonl",
              "/content/drive/MyDrive/thesis/multinli_dev_matched_gpt_uniEval.jsonl")

evaluate_data("/content/drive/MyDrive/thesis/multinli_dev_mismatched.jsonl",
              "/content/drive/MyDrive/thesis/multinli_dev_mismatched_gpt.jsonl",
              "/content/drive/MyDrive/thesis/multinli_dev_mismatched_gpt_uniEval.jsonl")

Evaluating coherence of 20000 samples !!!


100%|██████████| 2500/2500 [10:12<00:00,  4.08it/s]


Evaluating consistency of 20000 samples !!!


100%|██████████| 2640/2640 [10:20<00:00,  4.26it/s]


Evaluating fluency of 20000 samples !!!


...

100%|██████████| 2500/2500 [10:14<00:00,  4.07it/s]


Evaluating consistency of 20000 samples !!!


100%|██████████| 2587/2587 [10:04<00:00,  4.28it/s]


Evaluating fluency of 20000 samples !!!


100%|██████████| 2587/2587 [05:49<00:00,  7.41it/s]
